# **En este documento se desarrollan los modelos de asociación y clasificación seleccionados para el caso**



##Se cargan las bibliotecas

In [18]:
!pip install numpy
!pip install pandas

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


##Se carga la base con la que se trabajará y se renombrarán las columnas para su mejor visualización

In [20]:
tabla_violencias = pd.read_csv('/Tipo de violencias.csv')
tabla_violencias.head()

,Unnamed: 0,tipo_de_violencia_fisica,tipo_de_violencia_psicologica,tipo_de_violencia_sexual,tipo_de_violencia_economica_y_patrimonial,tipo_de_violencia_simbolica,tipo_de_violencia_domestica,cluster,cluster_predicted,dbscan_cluster
0,0,Si,Si,No,No,No,Si,1,2,0
1,1,Si,Si,No,No,No,Si,1,2,0
2,2,Si,Si,No,No,No,Si,1,2,0
3,3,No,Si,No,No,No,Si,1,1,1
4,4,Si,Si,No,No,No,Si,1,2,0


In [21]:
tabla_violencias = tabla_violencias.rename(columns={'tipo_de_violencia_fisica': 'TVF', 'tipo_de_violencia_psicologica': 'TVP','tipo_de_violencia_sexual': 'TVSE','tipo_de_violencia_economica_y_patrimonial': 'TVE','tipo_de_violencia_simbolica': 'TVSI','tipo_de_violencia_domestica': 'TVD'})

##Algoritmo Apriori

Se considera que para el objetivo de observar la co ocurrencia de los tipos de violencia es muy útil aplicar el algorimo Apriori que permite generar distintas reglas de asociación entre los diferentes tipos de violencia.

Se aplica este algoritmo con un **soporte de 0.6** para realizar el cruce de las variables con mayor correlación de acuerdo a lo observado en la descripción de la base Tipo de violencias.

In [22]:
# prompt: Mediante el marco de datos tabla_violencias: apriori

from mlxtend.frequent_patterns import apriori, association_rules

# Assuming 'tabla_violencias' is a Pandas DataFrame with categorical columns
# Convert categorical columns to boolean (True/False) for Apriori
tabla_violencias_encoded = tabla_violencias.apply(lambda x: x == 'Si')

# Apply Apriori algorithm
frequent_itemsets = apriori(tabla_violencias_encoded, min_support=0.5, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

print(rules) # Display the generated association rules


   antecedents consequents  antecedent support  consequent support   support  \
0        (TVF)       (TVP)            0.658532            0.949218  0.639301   
1        (TVP)       (TVF)            0.949218            0.658532  0.639301   
2        (TVF)       (TVD)            0.658532            0.908365  0.630579   
3        (TVD)       (TVF)            0.908365            0.658532  0.630579   
4        (TVP)       (TVD)            0.949218            0.908365  0.877909   
5        (TVD)       (TVP)            0.908365            0.949218  0.877909   
6   (TVF, TVP)       (TVD)            0.639301            0.908365  0.614764   
7   (TVF, TVD)       (TVP)            0.630579            0.949218  0.614764   
8   (TVP, TVD)       (TVF)            0.877909            0.658532  0.614764   
9        (TVF)  (TVP, TVD)            0.658532            0.877909  0.614764   
10       (TVP)  (TVF, TVD)            0.949218            0.630579  0.614764   
11       (TVD)  (TVF, TVP)            0.

##Análisis de los resultados del algoritmo Apriori

Se reconoce como importante, dados los parámetros antes mencionados, el resultado de la asociación entre el Tipo de violencia doméstica y el Tipo de violencia psicológica, dados los siguientes resultados:

| antecedents | consequents | antecedent support | consequent support | support | confidence | lift | leverage | conviction | zhangs_metric |
|---|---|---|---|---|---|---|---|---|---|
| (TVP) | (TD) | 0.949218 | 0.908365 | 0.877909 | 0.924876 | 1.018177 | 0.015673 | 1.219786 | 0.351550 |
| (TD) | (TVP) | 0.908365 | 0.949218 | 0.877909 | 0.966472 | 1.018177 | 0.015673 | 1.514608 | 0.194820 |

Aquí se observa que el soporte es del 0.87, con una confianza del 0.92 y 0.96 dependiendo qué variable se disponga como antecedente.


##Clasificación por K-means

In [23]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

# Select features for clustering (excluding target and irrelevant columns)
features = tabla_violencias[['TVF', 'TVP', 'TVSE', 'TVE', 'TVSI', 'TVD']]

# Convert categorical features to numerical using one-hot encoding
features_encoded = pd.get_dummies(features)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features_encoded, tabla_violencias['cluster'], test_size=0.2, random_state=42)

# Create and fit k-means model
kmeans = KMeans(n_clusters=3, random_state=0)  # Assuming 3 clusters based on existing 'cluster' column
kmeans.fit(X_train)

# Predict clusters for test set
y_pred = kmeans.predict(X_test)

# Evaluate model performance (example using accuracy)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Accuracy: 0.4533169533169533


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Assuming 'tabla_violencias' is your DataFrame and you want to use columns 'TVF' to 'TVD' for clustering
X = pd.get_dummies(tabla_violencias.loc[:, 'TVP':'TVD'])

# Se inicializa Kmeans con 3 clusters
kmeans = KMeans(n_clusters=8)

# Fit the model to your data
kmeans.fit(X)

# Get the cluster labels
labels = kmeans.labels_

# Calculate the Silhouette Score
score = silhouette_score(X, labels)

print(score)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


##Resultados por K-means

En base a los resultados obtenidos cabe mencionar que el resultado del modelo depende de la cantidad de clusters que se quieran conformar.
Así, al proponer un modelo con 3 clusters el valor del accuracy es de 0.45; mientras que en el modelo con 8 clusters el score se incrementa a 0.82.
Cabe mencionar que el tiempo de ejecución para esta cantidad de datos es cercano a los 2 minutos.

In [ ]:
##Clasificación por MiniBatch

In [ ]:
# prompt: Mediante el marco de datos tabla_violencias: minibatch

from sklearn.cluster import MiniBatchKMeans
tabla_violencias_encoded = tabla_violencias.apply(lambda x: x == 'Si')

clustering = MiniBatchKMeans(n_clusters=5, random_state=0).fit(tabla_violencias_encoded) # Perform minibatch k-means clustering with 5 clusters
score = silhouette_score(tabla_violencias_encoded, clustering.labels_)
print(score)

##Resultado por MiniBatch
En base a los resultados obtenidos se sostiene que el resultado del modelo depende de la cantidad de clusters que se quieran conformar. En el caso de la construcción de un modelo de Minibatch con 5 clusters el resultado del score es de 0.51, lo que no representa un buen resultado para el modelo.

Cabe mencionar que el tiempo de ejecución para esta cantidad de datos también es cercano a los 2 minutos.

##Otra opción de aprendizaje no supervisado
Se intentó usar el modelo de Affinity Propagation pero no se pudo realizar debido a la insuficiencia de memoria RAM